In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import datetime
import time

In [ ]:
"""
read data (processed_data)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("processed_data/index/"+filename))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("processed_data/stocks/"+filename))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
max0day = int(stocks[0].loc[stocks[0].iloc[:,1]==0].Date.max())

#stocks[0].iloc[max0day:,1:5].plot()
#plt.show()
data = stocks[0].iloc[max0day+1:].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [2]:
"""
read data (original)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("data/index/"+filename).fillna(0))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("data/stocks/"+filename).fillna(0))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
data = stocks[0].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [3]:
train = train.values
test = test.values

In [12]:
train_x = np.array([train[i:i+365] for i in range(len(train)-365)])[:, np.newaxis]
train_y = np.array([train[i+365] for i in range(len(train)-365)])[:, np.newaxis]

test_x = np.array([test[i:i+365] for i in range(len(test)-365)])[:, np.newaxis]
test_y = np.array([test[i+365] for i in range(len(test)-365)])[:, np.newaxis]


# print(train_x)
# print(len(train_x))
# print(train_y)
# print(len(train_y))

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(train, train, length=365, batch_size=128)

#x, y = data_gen[0]
#print(x)
#print(len(x))
#print(y)
#print(len(y))

In [11]:
"""
LSTM
"""
model = tf.keras.Sequential()
#model.add(layers.BatchNormalization())
# (i)
model.add(layers.LSTM(128, return_sequences=True, dropout=.5,
                     input_shape=(None, 365)))
model.add(layers.BatchNormalization())
model.add(layers.LSTM(64), dropout=.5)
#model.add(layers.Flatten())
model.add(layers.Dense(1))
#model.add(layers.CuDNNLSTM(128))
# (ii)
#model.add(layers.Bidirectional(layers.LSTM(128), return_sequences=True))
#model.add(layers.Bidirectional(layers.LSTM(64)))

adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam)

model.build()
print(model.summary())

model.fit(train_x, train_y,
          batch_size=128, 
          shuffle = True,
          epochs=200)

# model.fit_generator(data_gen, 
#                     epochs=200, 
#                     shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, None, 128)         252928    
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, None, 128)         512       
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 302,913
Trainable params: 302,657
Non-trainable params: 256
_________________________________________________________________
None
Epoch 1/200
8648/8648 [==============================] - 5s 541us/sample - loss: 1268.8553
Epoch 2/200
8648/8648 [==============================] - 1s 105us/sample - loss: 1004.3249
Epoch 3/200
8648/8648 [==============================] - 1s 110us/sample - loss

8648/8648 [==============================] - 1s 109us/sample - loss: 417.5308
Epoch 82/200
8648/8648 [==============================] - 1s 108us/sample - loss: 432.4061
Epoch 83/200
8648/8648 [==============================] - 1s 110us/sample - loss: 422.1145
Epoch 84/200
8648/8648 [==============================] - 1s 109us/sample - loss: 431.4750
Epoch 85/200
8648/8648 [==============================] - 1s 110us/sample - loss: 415.1862
Epoch 86/200
8648/8648 [==============================] - 1s 110us/sample - loss: 411.4411
Epoch 87/200
8648/8648 [==============================] - 1s 107us/sample - loss: 425.7325
Epoch 88/200
8648/8648 [==============================] - 1s 110us/sample - loss: 418.1063
Epoch 89/200
8648/8648 [==============================] - 1s 132us/sample - loss: 426.9664
Epoch 90/200
8648/8648 [==============================] - 1s 117us/sample - loss: 421.5519
Epoch 91/200
8648/8648 [==============================] - 1s 133us/sample - loss: 414.4013
Epoch 92/200

8648/8648 [==============================] - 1s 111us/sample - loss: 391.2664
Epoch 171/200
8648/8648 [==============================] - 1s 111us/sample - loss: 399.3784
Epoch 172/200
8648/8648 [==============================] - 1s 111us/sample - loss: 410.4397
Epoch 173/200
8648/8648 [==============================] - 1s 119us/sample - loss: 378.0418
Epoch 174/200
8648/8648 [==============================] - 1s 117us/sample - loss: 399.8706
Epoch 175/200
8648/8648 [==============================] - 1s 130us/sample - loss: 415.2540
Epoch 176/200
8648/8648 [==============================] - 1s 114us/sample - loss: 406.6279
Epoch 177/200
8648/8648 [==============================] - 1s 113us/sample - loss: 421.1325
Epoch 178/200
8648/8648 [==============================] - 1s 116us/sample - loss: 400.6630
Epoch 179/200
8648/8648 [==============================] - 1s 117us/sample - loss: 387.1888
Epoch 180/200
8648/8648 [==============================] - 1s 114us/sample - loss: 379.3785
Ep

In [15]:
model.evaluate(test_x, test_y)

365/365 [==============================] - 0s 140us/sample - loss: 18474.4372


18474.437248501712

In [ ]:
def smape_kun(y_true, y_pred):
    return np.mean((np.abs(np.array(y_pred) - np.array(y_true)) * 200/ (np.abs(y_pred) + np.abs(y_true))))
error2 = smape_kun(test_ar, predictions)
print('Symmetric mean absolute percentage error: %.3f' % error2)

In [ ]:
#plt.figure(figsize=(12,7))
%matplotlib qt

#plt.plot(data, 'green', color='blue', label='Training Data')
plt.plot(test.index, predictions, 
         color='green', 
         marker=',', #'o', 
         linestyle='dashed', 
         label='Predicted Price')
plt.plot(test.index, test, color='red', label='Actual Price')
plt.title('Prices Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
#plt.xticks(np.arange(0,1857, 300), stocks[0]['Date'][0:1857:300])
#plt.xticks(np.arange(max0day+1,len(data), 300), stocks[0]['Date'][max0day+1:len(data):300])
plt.legend()

The graph shows that the trained model just picks the same price of today as the price of the next day.